In [1]:
import pandas as pd
import mne
import numpy as np
from datetime import datetime
from datetime import timedelta
import Helper
import os
import json
from pathlib import Path

In [2]:
FMIN = 2.
FMAX = 30.
SAMPLING_RATE = 200 # eeg sample rate
TIME_OFFSET = 0

delta_band = [0.5, 4]
theta_band = [4, 8] 
alpha_band = [8,14]
beta_band = [14,30]
gamma_band = [30, 58]

all_ch = ['ch1','ch2','ch3','ch4']
# PICO_CHANNELS_ORDER = ['Pz', 'FCz', 'FC3', 'Oz']
PICO_CHANNELS_ORDER = ['FC3', 'FCz', 'Pz', 'Oz']

In [3]:
def filterEeg(eeg_data):
    for ch in all_ch:
        eeg_data[ch] = mne.filter.filter_data(eeg_data[ch].astype(float).to_numpy(), sfreq=SAMPLING_RATE, l_freq=FMIN, h_freq=FMAX, verbose=False)
        eeg_data[ch] = mne.filter.notch_filter(eeg_data[ch].astype(float).to_numpy(), Fs=SAMPLING_RATE, freqs=np.arange(60, 100, 60), verbose=False, filter_length='auto', phase='zero')

    return eeg_data

In [4]:
# Data

def changeTimeOffset(timestampSeries):
    timestampSeries = pd.to_datetime(timestampSeries)
    timestampSeries = timestampSeries.apply(lambda x : x - timedelta(days=0, seconds=0, microseconds=0, milliseconds=TIME_OFFSET, minutes=0, hours=0, weeks=0))
    return timestampSeries

def cleanEegData(eegData):
    cleanCols = ['ch1','ch2','ch3','ch4', "timeStamp"]
    cleanEegData = eegData[cleanCols]
    
    cleanEegData['timeStamp'] = changeTimeOffset(cleanEegData['timeStamp'])

    return cleanEegData

In [5]:
delta_band = [0.5, 4]
theta_band = [4, 8] 
alpha_band = [8,14]
beta_band = [14,30]
gamma_band = [30, 58]

In [6]:
def getColumns():
    eegBands = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    cols = []
    for ch in PICO_CHANNELS_ORDER:
        for eegBand in eegBands:
            cols.append(f"{ch}_{eegBand}")

    return cols        

def calEegFeatures(eeg_data):
    eegFeature_df = pd.DataFrame(columns=getColumns())
    # band power
    for ch, ach in zip(all_ch, PICO_CHANNELS_ORDER):
        # band
        delta_bandpower = Helper.bandpower(eeg_data[ch], SAMPLING_RATE, delta_band, window_sec = len(eeg_data[ch])/SAMPLING_RATE, relative=True)
        theta_bandpower = Helper.bandpower(eeg_data[ch], SAMPLING_RATE, theta_band, window_sec = len(eeg_data[ch])/SAMPLING_RATE, relative=True)
        alpha_bandpower = Helper.bandpower(eeg_data[ch], SAMPLING_RATE, alpha_band, window_sec = len(eeg_data[ch])/SAMPLING_RATE, relative=True)
        beta_bandpower = Helper.bandpower(eeg_data[ch], SAMPLING_RATE, beta_band, window_sec = len(eeg_data[ch])/SAMPLING_RATE, relative=True)
        gamma_bandpower = Helper.bandpower(eeg_data[ch], SAMPLING_RATE, gamma_band, window_sec = len(eeg_data[ch])/SAMPLING_RATE, relative=True)

        eegFeature_df[f'{ach}_delta'] = [delta_bandpower]
        eegFeature_df[f'{ach}_theta'] = [theta_bandpower]
        eegFeature_df[f'{ach}_alpha'] = [alpha_bandpower]
        eegFeature_df[f'{ach}_beta'] = [beta_bandpower]
        eegFeature_df[f'{ach}_gamma'] = [gamma_bandpower]

    return eegFeature_df


In [7]:
def set_label(dataInfo):
    addictLabel = int(dataInfo[0][0] == 'A')

    if dataInfo[3] == "PreTest":
        state = 0
    elif dataInfo[3] == "VRTest":
        state = 1
    else:
        state = 2
    
    stateLabel = state

    # label = {'addictLabel' : addictLabel, 'stateLabel':stateLabel, 'trainingCount': trainingCount}
    label = {'addictLabel' : addictLabel, 'stateLabel': stateLabel}
    
    return label

# Processing input

In [ ]:
# columns = ['Timestamp', 'ch1', 'ch2', 'ch3', 'ch4']
# csv_file = "NormalData/test08/VR_1_PostTest_EGG.csv"
# eeg_data = pd.read_csv(csv_file, header = None, names = columns)
# # print(eeg_data[0:5])
# eeg_data = cleanEegData(eeg_data)
# # print(eeg_data[0:5])
# eeg_data = filterEeg(eeg_data)

# result = pd.DataFrame(columns=getColumns())

# EEGfeatures = calEegFeatures(eeg_data)
# result = pd.concat([result, EEGfeatures], axis = 0)

In [22]:
# RestingState
# path = 'C:/Users/Jackson/OneDrive/LAB/資料/壓力訓練/正式收案/RestingState'
path = 'C:/Users/Jackson/OneDrive/LAB/資料/壓力訓練/正式收案/測試用'
data_path = Path(path)
data_path = [x for x in data_path.iterdir() if x.is_dir()]
all_data = []
columns = ['timeStamp', 'ch1', 'ch2', 'ch3', 'ch4']
result = pd.DataFrame(columns=getColumns())
for fd in data_path:
    # print(fd)
    eeg = Path(list(fd.rglob('*eeg.json'))[0], lines=True)
    # print(eeg)
    strData = open(eeg,"r",encoding='utf-8').read()
    listData = strData.split("\n")
    listData = listData[:len(listData)-1]
    eeg_data = []
    for data in listData:
        temp = []
        temp.append(json.loads(data)['timeStamp'].replace('+08:00', ''))
        temp.append(json.loads(data)['ch1'])
        temp.append(json.loads(data)['ch2'])
        temp.append(json.loads(data)['ch3'])
        temp.append(json.loads(data)['ch4'])
        eeg_data.append(temp)
    eeg_data = pd.DataFrame(eeg_data,columns=['timeStamp', 'ch1', 'ch2', 'ch3', 'ch4'])
    # print(eeg_data.columns)
    eeg_data = cleanEegData(eeg_data)
    eeg_data = filterEeg(eeg_data)
    EEGfeatures = calEegFeatures(eeg_data)
    result = pd.concat([result, pd.DataFrame(EEGfeatures, index=[0])], axis=0, ignore_index=True)

# path = 'C:/Users/Jackson/OneDrive/LAB/資料/壓力訓練/測試資料/RestingState/202403150520_N06_DPost/N06_DPost_eeg.json'

c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = f

In [23]:
result.to_csv('C:/Users/Jackson/Downloads/result.csv')
result

,FC3_delta,FC3_theta,FC3_alpha,FC3_beta,FC3_gamma,FCz_delta,FCz_theta,FCz_alpha,FCz_beta,FCz_gamma,Pz_delta,Pz_theta,Pz_alpha,Pz_beta,Pz_gamma,Oz_delta,Oz_theta,Oz_alpha,Oz_beta,Oz_gamma
0,0.722548,0.133796,0.063596,0.053424,0.005466,0.843805,0.082274,0.032391,0.026149,0.002596,0.720347,0.135118,0.064314,0.054428,0.00566,0.787037,0.103366,0.047435,0.040145,0.00417


In [11]:
def data_prepro(path1, path2):
    strData = open(path1,"r",encoding='utf-8').read()
    listData = strData.split("\n")
    listData = listData[:len(listData)-1]
    columns = ['timeStamp', 'ch1', 'ch2', 'ch3', 'ch4']
    result = pd.DataFrame(columns=getColumns())
    eeg_data = []
    for data in listData:
        temp = []
        temp.append(json.loads(data)['timeStamp'].replace('+08:00', ''))
        temp.append(json.loads(data)['ch1'])
        temp.append(json.loads(data)['ch2'])
        temp.append(json.loads(data)['ch3'])
        temp.append(json.loads(data)['ch4'])
        eeg_data.append(temp)
    eeg_data = pd.DataFrame(eeg_data,columns=['timeStamp', 'ch1', 'ch2', 'ch3', 'ch4'])
    strData = open(path2, 'r', encoding='utf-8').read()
    listData = strData.split('\n')
    listData = listData[:len(listData)-1]
    game = []
    for data in listData:
        temp = []
        temp.append(json.loads(data)['start_time'].replace('+08:00', ''))
        temp.append(json.loads(data)['end_time'].replace('+08:00', ''))
        game.append(temp)
    game = pd.DataFrame(game,columns=['start_time', 'end_time'])
    phase = [[] for i in range(3)]
    timer_data = np.array(game[['start_time', 'end_time']])
    for tv in range(len(eeg_data)):
        time = pd.to_datetime(eeg_data['timeStamp'][tv])
        if time < pd.to_datetime(timer_data[0][1]): #第一圈
            phase[0].append(eeg_data.loc[tv].values.flatten().tolist())
        elif time < pd.to_datetime(timer_data[1][1]) and time >= pd.to_datetime(timer_data[1][0]): #第二圈
            phase[1].append(eeg_data.loc[tv].values.flatten().tolist())
        elif time < pd.to_datetime(timer_data[2][1]) and time >= pd.to_datetime(timer_data[2][0]): #第三圈
            phase[2].append(eeg_data.loc[tv].values.flatten().tolist())
    for i in range(len(phase)):
        phase[i] = pd.DataFrame(phase[i],columns=['timeStamp', 'ch1', 'ch2', 'ch3', 'ch4'])
    for p in phase:
        eeg_data = cleanEegData(p)
        eeg_data = filterEeg(eeg_data)
        EEGfeatures = calEegFeatures(eeg_data)
        result = pd.concat([result, pd.DataFrame(EEGfeatures, index=[0])], axis=0, ignore_index=True)
    return result

In [20]:
# WarCar
# path1 = 'C:/Users/Jackson/OneDrive/LAB/資料/壓力訓練/測試資料/WarCar/202403150424_N06_D_Stroop/N06_D_Stroop_eeg.json'
# path2 = 'C:/Users/Jackson/OneDrive/LAB/資料/壓力訓練/測試資料/WarCar/202403150424_N06_D_Stroop/N06_D_Stroop_Game.json'
# path1 = 'C:/Users/Jackson/OneDrive/LAB/資料/壓力訓練/測試資料/WarCar/202403150456_N06_D_CPT/N06_D_CPT_eeg.json'
# path2 = 'C:/Users/Jackson/OneDrive/LAB/資料/壓力訓練/測試資料/WarCar/202403150456_N06_D_CPT/N06_D_CPT_Game.json'
# path = 'C:/Users/Jackson/OneDrive/LAB/資料/壓力訓練/正式收案/WarCar'
path = 'C:/Users/Jackson/OneDrive/LAB/資料/壓力訓練/正式收案/分開算'
data_path = Path(path)
cpt = [x for x in data_path.rglob('*CPT') if x.is_dir()]
stroop = [x for x in data_path.rglob('*Stroop') if x.is_dir()]
cpt_all_data = pd.DataFrame(columns=getColumns())
stroop_all_data = pd.DataFrame(columns=getColumns())
temp = pd.DataFrame(columns=getColumns())
for fd in cpt:
    print(fd)
    eeg = Path(list(fd.rglob('*eeg.json'))[0], lines=True)
    game = Path(list(fd.rglob('*Game.json'))[0], lines=True)
    temp = data_prepro(eeg, game)
    cpt_all_data = pd.concat([cpt_all_data, pd.DataFrame(temp)], axis=0, ignore_index=True)
for fd in stroop:
    print(fd)
    eeg = Path(list(fd.rglob('*eeg.json'))[0], lines=True)
    game = Path(list(fd.rglob('*Game.json'))[0], lines=True)
    temp = data_prepro(eeg, game)
    stroop_all_data = pd.concat([stroop_all_data, pd.DataFrame(temp)], axis=0, ignore_index=True)

C:\Users\Jackson\OneDrive\LAB\資料\壓力訓練\正式收案\分開算\202405020856_G14_D_Stroop


c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = f

C:\Users\Jackson\OneDrive\LAB\資料\壓力訓練\正式收案\分開算\202405070852_G15_D_Stroop


c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = f

C:\Users\Jackson\OneDrive\LAB\資料\壓力訓練\正式收案\分開算\202405081130_G16_D_Stroop


c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = f

C:\Users\Jackson\OneDrive\LAB\資料\壓力訓練\正式收案\分開算\202405130421_G17_D_Stroop


c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = f

In [14]:
cpt_all_data.to_csv('C:/Users/Jackson/Downloads/result.csv')
cpt_all_data

,FC3_delta,FC3_theta,FC3_alpha,FC3_beta,FC3_gamma,FCz_delta,FCz_theta,FCz_alpha,FCz_beta,FCz_gamma,Pz_delta,Pz_theta,Pz_alpha,Pz_beta,Pz_gamma,Oz_delta,Oz_theta,Oz_alpha,Oz_beta,Oz_gamma
0,0.734574,0.127935,0.063822,0.052494,0.003911,0.718487,0.152833,0.067314,0.046642,0.003744,0.736955,0.138546,0.065129,0.051405,0.004032,0.832940,0.095088,0.023815,0.031906,0.002428
1,0.718552,0.146481,0.068206,0.055479,0.004512,0.782164,0.122374,0.048098,0.039639,0.003212,0.685712,0.163368,0.082467,0.059278,0.004586,0.882847,0.058392,0.026245,0.024903,0.002499
2,0.721573,0.146487,0.068518,0.053275,0.004621,0.720707,0.163873,0.060330,0.048096,0.003734,0.717433,0.149353,0.067956,0.053633,0.004394,0.780879,0.122811,0.048236,0.040726,0.003438
3,0.580727,0.263283,0.051812,0.082005,0.005985,0.756948,0.144622,0.034216,0.046923,0.002889,0.752880,0.154582,0.033356,0.045670,0.004276,0.729326,0.120214,0.057301,0.042908,0.003477
4,0.489831,0.306896,0.068418,0.095824,0.007715,0.578108,0.247129,0.061382,0.081023,0.006517,0.688444,0.178665,0.043347,0.062620,0.004920,0.777822,0.120622,0.055763,0.042803,0.003374
5,0.526425,0.277380,0.084190,0.098126,0.008266,0.590708,0.230804,0.070471,0.093733,0.007768,0.617616,0.215037,0.064602,0.089577,0.007480,0.757326,0.128497,0.060949,0.045827,0.003516
6,0.670684,0.175505,0.081695,0.063115,0.005769,0.686691,0.167714,0.078022,0.059442,0.005514,0.705525,0.149440,0.071252,0.055496,0.004992,0.681033,0.168175,0.078517,0.060783,0.005583
7,0.589740,0.239164,0.067869,0.084294,0.006686,0.839498,0.081201,0.020883,0.040147,0.002062,0.855719,0.078162,0.019873,0.034051,0.001801,0.856705,0.079719,0.020493,0.033068,0.002094
8,0.549402,0.273237,0.086406,0.086689,0.007689,0.620618,0.230897,0.071137,0.074308,0.006386,0.651857,0.206707,0.068389,0.068626,0.005767,0.619930,0.231812,0.074129,0.074764,0.006368
9,0.717650,0.140872,0.074337,0.053810,0.004873,0.688060,0.171263,0.076729,0.051785,0.004758,0.721947,0.148006,0.072099,0.049182,0.003757,0.726419,0.151351,0.066575,0.046377,0.003792


In [21]:
stroop_all_data.to_csv('C:/Users/Jackson/Downloads/result.csv')
stroop_all_data

,FC3_delta,FC3_theta,FC3_alpha,FC3_beta,FC3_gamma,FCz_delta,FCz_theta,FCz_alpha,FCz_beta,FCz_gamma,Pz_delta,Pz_theta,Pz_alpha,Pz_beta,Pz_gamma,Oz_delta,Oz_theta,Oz_alpha,Oz_beta,Oz_gamma
0,0.565757,0.240670,0.084013,0.096044,0.007103,0.624025,0.200014,0.080928,0.083886,0.006278,0.656449,0.190654,0.071815,0.072715,0.005426,0.556372,0.245557,0.088320,0.098900,0.007336
1,0.589454,0.232461,0.081839,0.083349,0.006209,0.655999,0.189376,0.076792,0.073438,0.004891,0.661151,0.187897,0.071712,0.067049,0.004897,0.595794,0.225806,0.079773,0.081181,0.005930
2,0.633646,0.202993,0.076760,0.076978,0.005528,0.663840,0.187001,0.068866,0.063746,0.005081,0.671301,0.187135,0.064725,0.065790,0.004638,0.633957,0.203692,0.074444,0.075823,0.005430
3,0.697567,0.154577,0.077610,0.060565,0.004966,0.796952,0.130235,0.033370,0.031314,0.005153,0.779444,0.134120,0.041111,0.036736,0.005479,0.733896,0.139131,0.063194,0.052246,0.003718
4,0.675811,0.167251,0.080858,0.064158,0.005118,0.690823,0.164342,0.074475,0.062193,0.005097,0.669666,0.168588,0.087248,0.069564,0.005173,0.706137,0.155604,0.074090,0.058655,0.004437
5,0.706067,0.153545,0.079919,0.061501,0.005030,0.709893,0.146195,0.078992,0.061613,0.004637,0.699762,0.151899,0.082939,0.063889,0.005048,0.678439,0.163992,0.080782,0.059998,0.004459
6,0.743931,0.121904,0.073376,0.052615,0.004117,0.770388,0.112893,0.049382,0.044489,0.003383,0.723007,0.141324,0.053849,0.059509,0.004695,0.841777,0.080490,0.030634,0.034027,0.002640
7,0.722251,0.139860,0.075610,0.053887,0.004172,0.711195,0.154664,0.059703,0.056022,0.003514,0.633384,0.199365,0.077103,0.072841,0.005618,0.743067,0.141518,0.054582,0.044423,0.003315
8,0.725944,0.143956,0.069260,0.054237,0.004251,0.758410,0.118152,0.044494,0.046214,0.003807,0.692748,0.163969,0.061818,0.061756,0.004602,0.692823,0.165307,0.063299,0.060771,0.004697
9,0.644178,0.208802,0.051423,0.071211,0.005477,0.675014,0.193630,0.058876,0.057930,0.004054,0.728211,0.151884,0.054077,0.048052,0.004000,0.626307,0.203056,0.099733,0.063138,0.005583


In [ ]:
workDir = 'D:/LAB/放鬆-三總/Relaxing/分析'
os.chdir(workDir)
inputPath = "Data/EEG/"
data_name = ['data_name']
lable_names = ['addictLabel', 'stateLabel']
columns = ['Timestamp', 'ch1', 'ch2', 'ch3', 'ch4']

dataList = os.listdir(inputPath)
name = pd.DataFrame(columns = data_name)
result = pd.DataFrame(columns=getColumns())
lable = pd.DataFrame(columns=lable_names)

for dataName in dataList:
    # A1_VR_1_PreTest_EEG
    dataInfo = dataName[:-4].split('_')
    if dataInfo[0] == 'A14' or dataInfo[0] == 'A17':
        continue
    
    eeg_data = pd.read_csv("%s%s"%(inputPath, dataName), header = None, names=columns)
    # print(eeg_data[0:5])
    eeg_data = cleanEegData(eeg_data)
    eeg_data = filterEeg(eeg_data)
    # print(eeg_data[0:5])

    # if (dataInfo[0] == 'N01' or dataInfo[0] == 'N02' or dataInfo[0] == 'N03' or dataInfo[0] == 'N04' or dataInfo[0] == 'N06' or dataInfo[0] == 'N08'
    #     or dataInfo[0] == 'A01' or dataInfo[0] == 'A02'):
    #     SAMPLING_RATE = 60 # eeg sample rate
    # else:
    #     SAMPLING_RATE = 200 # eeg sample rate
    
    if (dataInfo[3] == 'PreTest'):
        # print(eeg_data.iloc[0].Timestamp, eeg_data.iloc[0].Timestamp+timedelta(seconds=0.5))
        eeg_basedata = eeg_data[eeg_data.Timestamp <= eeg_data.iloc[0].Timestamp + timedelta(seconds=1.5)] # 1s 有些 baseline 會為0(sample rate = 60)
        # print(eeg_basedata.iloc[-1])
        eeg_data = eeg_data[eeg_data.Timestamp > eeg_data.iloc[0].Timestamp + timedelta(seconds=1.5)]

        EEGbasefeatures = calEegFeatures(eeg_basedata)
        datanameDF = pd.DataFrame({"data_name" : ["%s_%s"%(dataInfo[0], 'baseline')]})
        name = pd.concat([name, datanameDF], axis=0, ignore_index=True)
        result = pd.concat([result, pd.DataFrame(EEGbasefeatures, index=[0])], axis=0, ignore_index=True)
        lable = pd.concat([lable, pd.DataFrame({'addictLabel' : int(dataInfo[0][0] == 'A'), 'stateLabel': 3}, index=[0])], axis=0, ignore_index=True)

    EEGfeatures = calEegFeatures(eeg_data)

    # features = feature_Cal(data, sampleRate, 'RESdata')
    datanameDF = pd.DataFrame({"data_name" : ["%s_%s"%(dataInfo[0], dataInfo[3][:-4])]})
    name = pd.concat([name, datanameDF], axis=0, ignore_index=True)
    result = pd.concat([result, pd.DataFrame(EEGfeatures, index=[0])], axis=0, ignore_index=True)
    lable = pd.concat([lable, pd.DataFrame(set_label(dataInfo), index=[0])], axis=0, ignore_index=True)
    
result = name.join(result)
result = result.join(lable)


In [ ]:
result.to_csv('Feature/EEG.csv')
result

In [ ]:
baselineData = result[result.stateLabel == 3]
EEGfData = result[result.stateLabel != 3]
for index, row in EEGfData.iterrows():
    # print(row.data_name)
    dn = row.data_name.split('_')
    # print(baselineData.data_name.str.startswith(dn[0]))
    bld = baselineData[baselineData.data_name.str.startswith(dn[0])]
    # print(bld)
    eegFeature_df = pd.DataFrame(columns=getColumns())
    for c in eegFeature_df:
        row[c] = (row[c] - bld[c]) / bld[c] * 100
        # print(row[c])
        EEGfData.at[index, c] = row[c].iloc[0]
    # print(row)
    # print('______________________')
EEGfData.to_csv('Feature/EEG_PCR.csv')
EEGfData

In [ ]:
# preResult = result[result['data_name'].str.endswith('_Pre')].reset_index(drop=True)
# # preResult.to_csv('Feature/EEG_PreF.csv')
# postResult = result[result['data_name'].str.endswith('_Post')].reset_index(drop=True)
# # postResult
# ppResult = pd.concat([preResult, postResult], axis=0, ignore_index=True)
# ppResult
# # postResult.to_csv('Feature/EEG_PostF.csv')
# ppResult.to_csv('Feature/EEG_PrePost.csv')